In [1]:
AGENT_NAME = '20 bin PPO 500 results/default_PPO_citylearn_challenge_2022_phase_2_Building_6_20_bins_500.zip'
DATASET_NAME = 'citylearn_challenge_2022_phase_2' #only action is electrical storage

In [2]:
from stable_baselines3 import SAC, PPO
from stable_baselines3.common.env_checker import check_env
import gym

from citylearn.data import DataSet

import numpy as np

from KBMproject import ATLA
import KBMproject.utilities as utils

%matplotlib inline

In [3]:
agent = PPO.load(path=f"{AGENT_NAME}")

env = utils.make_discrete_env(schema=DataSet.get_schema(DATASET_NAME),  
                        action_bins=agent.action_space[0].n,
                        seed=42)

adv_env = ATLA.AdversaryATLAWrapper(env=env,
                                    adv_reward=ATLA.NormScaleReward(env, np.inf),
                                    victim=agent)
check_env(adv_env)

c:\Users\Broda-Milian\anaconda3\envs\CityLearnART\lib\site-packages\stable_baselines3\common\env_checker.py:384: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn(


In [4]:
policy_kwargs = dict(net_arch=[256, 256])
agent = SAC('MlpPolicy', 
            adv_env,
            device='cuda',
            policy_kwargs=policy_kwargs,
            #tensorboard_log='logs/Phase1/SAC/',
            )

Instead of saving the most recent obs, can I use env.buf_obs[-1]?

In [41]:
agent_dict = vars(agent)

In [68]:
keys = ['device','gamma','learning_rate']
{k: agent_dict[k] for k in keys if k in agent_dict}

{'device': device(type='cuda'), 'gamma': 0.99, 'learning_rate': 0.0003}

In [32]:
adv_r_dict = vars(agent.env.get_attr('adv_reward')[0])

In [66]:
agent.env.get_attr('adv_reward')[0].__class__.__name__

'NormScaleReward'

In [33]:
adv_r_dict

{'ord': inf, 'exp': 1, 'max_norm': 1.0}

In [34]:
adv_r_dict = {'adversarial reward ' + k: v for k, v in adv_r_dict.items()}

In [35]:
adv_r_dict

{'adversarial reward ord': inf,
 'adversarial reward exp': 1,
 'adversarial reward max_norm': 1.0}

In [65]:
agent.get_env().env_is_wrapped(ATLA.AdversaryATLAWrapper)

[True]

In [63]:
from stable_baselines3.common.env_util import is_wrapped
is_wrapped(agent.get_env().unwrapped, ATLA.AdversaryATLAWrapper)

False

In [47]:
is_wrapped(adv_env, ATLA.AdversaryATLAWrapper)

True

In [5]:
agent.learn(total_timesteps=10,
            progress_bar=True,
            callback=ATLA.AdvDistanceTensorboardCallback())

Output()